# Image Classifier (8 layers)

In [28]:
import numpy
from keras.layers import Conv2D,Flatten,MaxPooling2D,Dense,Dropout,Activation
from keras.models import Model,Sequential
from keras.optimizers import Adam
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator

### Limiting GPU Usage 

In [29]:
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 1.0
set_session(tf.Session(config=config))

In [30]:
batch_size = 10
num_classes = 50
epochs = 50
data_augmentation = True

##  The Model
 
- For training we modified the famous **Caltech-256** to *50 Classes* for training the classifier on small scale.
- Then we have created a simple Convolutional neural network of 8 layers to train the data.
 

In [31]:
model =Sequential()
#block 1
model.add(Conv2D(32, (3, 3), padding='same',activation = 'relu',input_shape = (224,224,3)))
model.add(Conv2D(32, (3, 3),padding = 'valid',activation = 'relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))
#block 2
model.add(Conv2D(64, (3, 3), padding='same', activation = 'relu'))
model.add(Conv2D(64, (3, 3), padding='valid', activation ='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))
#block 3
model.add(Conv2D(128,(3,3), padding = 'same', activation = 'relu'))
model.add(Conv2D(128,(3,3), padding = 'valid', activation = 'relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))
#block 4
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [32]:
opt = Adam(lr=0.0001, decay=1e-6)
model.compile(loss='categorical_crossentropy',optimizer=opt,metrics=['accuracy'])

#### - Since we have modified the data, implementing the Data Augmentation on our data will give us better results.
#### - *Keras*  provides inbuilt function named _ImageDataGenerator_  for the data augmentation

In [33]:
# This is the augmentation configuration we will use for training

if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(x_train, y_train,batch_size=batch_size, epochs=epochs, validation_data=(x_test, y_test), shuffle=True)
    
else:
    print('Using real-time data augmentation.')
    train_datagen = ImageDataGenerator(
        rescale=1./255,# set rescaling factor (applied before any other transformation)
        rotation_range=20,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        zoom_range=0.2,  # set range for random zoom
        fill_mode='nearest',  # set mode for filling points outside the input boundaries
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False,  # randomly flip images
        data_format=None ) # image data format, either "channels_first" or "channels_last"
    

Using real-time data augmentation.


In [34]:
# This is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1./255)

In [35]:
# this is a generator that will read pictures found in
#subfolers of 'data/train', and indefinitely
#generate batches of augmented image data
train_generator = train_datagen.flow_from_directory(
        r'/home/ug2017/mec/17135096/faster_rcnn/Train_data',  # this is the target directory
        target_size=(224, 224),  # all images will be resized to 224x224
        batch_size=batch_size)

Found 7535 images belonging to 50 classes.


## Training the Classifier



In [ ]:
model.fit_generator(
        train_generator,
        steps_per_epoch= (7535 // batch_size)+1,
        epochs=epochs)

Epoch 1/50
  9/754 [..............................] - ETA: 34:01 - loss: 3.9493 - acc: 0.0222

### After fitting the data, we have obtained the accuracy near 60% on the *training dataset*. 
